<H1>Linear Programming Algebra</H1>

<H3>Solving Linear Systems</H3>
Let's start by first solving a linear system of equations. Consider the following system.

\begin{eqnarray}
2 x_1 + 1 x_2 + 1 x_3 &=& 4 \nonumber \\
4 x_1 - 6 x_2 + 0 x_3 &=& 2 \nonumber \\
-2 x_1 + 7 x_2 + 2 x_3 &=& 1 \nonumber
\end{eqnarray}

Let's first represent this system as a matrix so that we can perform row operations.

In [1]:
import pandas as pd
import numpy as np
A = pd.DataFrame([[2, 1, 1, 4], [4, -6, 0, 2], [-2, 7, 2, 1]],
                  dtype=np.float64,
                  columns=['x1', 'x2', 'x3', 'RHS'])
A

,x1,x2,x3,RHS
0,2.0,1.0,1.0,4.0
1,4.0,-6.0,0.0,2.0
2,-2.0,7.0,2.0,1.0


We systematically perform row operations until each equation has a single variable isolated. Consider $x_1$ in the first equation. We can first turn its coefficient into a 1.

In [2]:
A.ix[0] = A.ix[0]/2
A

,x1,x2,x3,RHS
0,1.0,0.5,0.5,2.0
1,4.0,-6.0,0.0,2.0
2,-2.0,7.0,2.0,1.0


Now, let's use the $x_1$ in the first equation to eliminate $x_1$ in the other two equations. We'll need to add $-4x_1$ to the second equation and $2x_1$ in the third.

In [3]:
A.ix[1] = A.ix[1] - 4*A.ix[0]
A.ix[2] = A.ix[2] + 2*A.ix[0]
A

,x1,x2,x3,RHS
0,1.0,0.5,0.5,2.0
1,0.0,-8.0,-2.0,-6.0
2,0.0,8.0,3.0,5.0


Let's write a method that will take in the row and column position of the variable and equation we are solving for, and performs the necessary row operations to eliminate that variable from all other equations.

In [4]:
def pivot(A, i, j):
    """ 
    Solve the ith equation for variable j, then eliminate
    variable j from all other equations.
    """
    m, n = A.shape
    A.ix[i] = A.ix[i]/A.ix[i, j]
    for row in range(m):
        if row == i:
            continue
        A.ix[row] = A.ix[row] - A.ix[row, j]*A.ix[i]

Let's test this method to see if it can replicate the row operations we performed manually.

In [5]:
A = pd.DataFrame([[2, 1, 1, 4], [4, -6, 0, 2], [-2, 7, 2, 1]],
                  dtype=np.float64,
                  columns=['x1', 'x2', 'x3', 'RHS'])
pivot(A, 0, 'x1')
A

,x1,x2,x3,RHS
0,1.0,0.5,0.5,2.0
1,0.0,-8.0,-2.0,-6.0
2,0.0,8.0,3.0,5.0


Good. Now let's solve for $x_2$ in the second equation and finally $x_3$ in the third.

In [6]:
pivot(A, 1, 'x2')
A

,x1,x2,x3,RHS
0,1.0,0.0,0.375,1.625
1,-0.0,1.0,0.250,0.750
2,0.0,0.0,1.000,-1.000


In [7]:
pivot(A, 2, 'x3')
A

,x1,x2,x3,RHS
0,1.0,0.0,0.0,2.0
1,-0.0,1.0,0.0,1.0
2,0.0,0.0,1.0,-1.0


<H3>Solving Linear Programs</H3>

Consider the following LP:
\begin{eqnarray}
\max_{x,y} && 6x + 4y = z \nonumber \\
\mbox{s.t.} && x + y \le 6 \nonumber \\
&& 2x + y \le 9 \nonumber \\
&& 2x + 3y \le 16 \nonumber \\
&& x, y \ge 0 \nonumber
\end{eqnarray}

Append slack variables and write the objective as $-z + 6x + 4y = 0$ to make a system of equations.

\begin{eqnarray}
\max_{x,y,s} && 6x + 4y + 0 s_1 + 0 s_2 + 0 s_3 = z\nonumber \\
\mbox{s.t.} && 1x + 1y + 1s_1 + 0 s_2 + 0 s_3 = 6 \nonumber \\
&& 2x + 1y + 0s_1 + 1s_2 + 0s_3 = 9 \nonumber \\
&& 2x + 3y + 0s_1 + 0s_2 + 1s_3 = 16 \nonumber \\
&& x, y, s_1, s_2, s_3 \ge 0 \nonumber
\end{eqnarray}

This system can now be treated as a matrix, on which we'll perform row operations.

\begin{eqnarray}
\left[ \begin{array}{rrrrrr|r}
z & x & y & s_1 & s_2 & s_3 & RHS \\
-1 & 6 & 4 & 0 & 0 & 0 & 0 \\
0 & 1 & 1 & 1 & 0 & 0 & 6 \\
0 & 2 & 1 & 0 & 1 & 0 & 9 \\
0 & 2 & 3 & 0 & 0 & 1 & 16
\end{array} \right] \nonumber
\end{eqnarray}

In [8]:
A = pd.DataFrame([[-1, 6, 4, 0, 0, 0, 0], 
                 [0, 1, 1, 1, 0, 0, 6], 
                 [0, 2, 1, 0, 1, 0, 9], 
                 [0, 2, 3, 0, 0, 1, 16]], 
                 dtype=np.float64,
                 columns=['z', 'x', 'y', 's1', 's2', 's3', 'RHS'])
A

,z,x,y,s1,s2,s3,RHS
0,-1.0,6.0,4.0,0.0,0.0,0.0,0.0
1,0.0,1.0,1.0,1.0,0.0,0.0,6.0
2,0.0,2.0,1.0,0.0,1.0,0.0,9.0
3,0.0,2.0,3.0,0.0,0.0,1.0,16.0


The simplex method always maintains a basic feasible solution which has exactly one non-zero (or basic) variable per constraint. With 3 constraints, we'll have 3 basic variables. The set of basic variables is called the basis.

The current objective is $z = 6x+4y$, with a current basic solution of $(s_1, s_2, s_3) = (6, 9, 16)$, and current objective value of $0$. We'll need to increase the value of either $x$ or $y$ in order to improve the solution.

We'll pick $x$ as it currently has the highest objective coefficient. We'll push the value of $x$ as high as possible given the constraints. For the first constraint, there is a $1x$ on the left and $6$ on the right, so $x$ can go as high as $6$ there. For the second constraint, $x$ can go as high as $9/2$, and for the third, as high as $8$. The second constraint is the most binding, so we'll solve for $x$ there and eliminate $x$ from the rest of the equations.

In [9]:
pivot(A, 2, 'x')
A

,z,x,y,s1,s2,s3,RHS
0,-1.0,0.0,1.0,0.0,-3.0,0.0,-27.0
1,0.0,0.0,0.5,1.0,-0.5,0.0,1.5
2,0.0,1.0,0.5,0.0,0.5,0.0,4.5
3,0.0,0.0,2.0,0.0,-1.0,1.0,7.0


The equation that represented the objective is now $-z + y - 3s_2 = - 27$, so the equivalent objective is $y - 3s_2 + 27$. The second equation was previously solved for $s_2$ but is now solved for $x$, so $x$ has entered the basis and $s_2$ has exited. The current basic feasible solution is $(s_1, x, s_3) = (1.5, 4.5, 7)$, with an objective value of $27$. But $y$ still has a positive coefficient in the objective so there is still potential for improvement.

Let's look at the constraints imposed on $y$. In the first constraint, $y$ can go up to $1.5/0.5=3$, in the second $4.5/0.5=9$, and in the third $7/2=3.5$. The first constraint is the most binding, so we'll solve for $y$ there. This should kick $s_1$ out of the basis in favor of $y$.

In [10]:
pivot(A, 1, 'y')
A

,z,x,y,s1,s2,s3,RHS
0,-1.0,0.0,0.0,-2.0,-2.0,0.0,-30.0
1,0.0,0.0,1.0,2.0,-1.0,0.0,3.0
2,0.0,1.0,0.0,-1.0,1.0,0.0,3.0
3,0.0,0.0,0.0,-4.0,1.0,1.0,1.0


Now the transformed objective is $z = -2s_1 - 2s_2 + 30$, and the basic feasible solution is $(x, y, s_3) = (3, 3, 1)$ with a corresponding objective value of $30$. The non-basic variables $s_1$ and $s_2$ can be set to zero, and we'll still have a feasible solution using the basic variables. Our transformed objective is equivalent to the original objective, but it's clear that unless we allow negative values (which we don't), the objective can never go above 30. Since we have a feasible solution with an objective value of 30, that solution is provably optimal.

In [1]:
suite = unittest.TestLoader().loadTestsFromTestCase(TestDiet)
unittest.TextTestRunner().run(suite)

NameError: name 'unittest' is not defined